# Loudness Calculation for Non-Stationary Signals

This notebook demonstrates how to use the `loudness_zwtv` feature to calculate time-varying loudness for audio signals using the Zwicker method (ISO 532-1:2017).

## Requirements

- wandas with loudness support
- numpy, matplotlib (for visualization)
- An audio file (or use generated signal)

In [ ]:
import wandas as wd
import numpy as np
import matplotlib.pyplot as plt

# Enable inline plotting
%matplotlib inline

## 1. Basic Usage

Generate a simple test signal and calculate its loudness.

In [ ]:
# Generate a 1 kHz sine wave at moderate level
signal = wd.generate_sin(freqs=[1000], duration=2.0, sampling_rate=48000)
signal = signal * 0.063  # Scale to ~70 dB SPL

# Calculate loudness
loudness = signal.loudness_zwtv()

# Print statistics
print(f"Signal shape: {signal.shape}")
print(f"Loudness shape: {loudness.shape}")
print(f"Mean loudness: {loudness.mean():.2f} sones")
print(f"Max loudness: {loudness.max():.2f} sones")
print(f"Min loudness: {loudness.min():.2f} sones")

In [ ]:
# Plot the loudness over time
loudness.plot(title="Time-varying Loudness (sones)")
plt.show()

## 2. Field Type Comparison

Compare loudness calculations for free field vs diffuse field.

In [ ]:
# Generate multi-frequency signal
signal = wd.generate_sin(freqs=[1000, 2000, 4000], duration=1.0, sampling_rate=48000)
signal = signal * 0.1

# Calculate for both field types
loudness_free = signal.loudness_zwtv(field_type="free")
loudness_diffuse = signal.loudness_zwtv(field_type="diffuse")

# Print comparison
print(f"Free field mean loudness: {loudness_free.mean():.2f} sones")
print(f"Diffuse field mean loudness: {loudness_diffuse.mean():.2f} sones")
print(f"Ratio (free/diffuse): {loudness_free.mean()/loudness_diffuse.mean():.2f}")

In [ ]:
# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(12, 8))
loudness_free.plot(ax=axes[0], title="Free Field Loudness")
loudness_diffuse.plot(ax=axes[1], title="Diffuse Field Loudness")
plt.tight_layout()
plt.show()

## 3. Amplitude Dependency

Demonstrate how loudness varies with signal amplitude.

In [ ]:
# Test at different amplitudes
amplitudes = [0.01, 0.03, 0.1, 0.3]  # Approximately 50, 60, 70, 80 dB SPL
loudness_values = []

for amp in amplitudes:
    test_signal = wd.generate_sin(freqs=[1000], duration=0.5, sampling_rate=48000)
    test_signal = test_signal * amp
    loud = test_signal.loudness_zwtv()
    mean_loudness = loud.mean()
    loudness_values.append(mean_loudness)
    print(f"Amplitude {amp:.3f}: {mean_loudness:.2f} sones")

In [ ]:
# Plot amplitude vs loudness
plt.figure(figsize=(10, 6))
plt.plot(amplitudes, loudness_values, 'o-', linewidth=2, markersize=8)
plt.xlabel('Signal Amplitude')
plt.ylabel('Mean Loudness (sones)')
plt.title('Loudness vs Amplitude')
plt.grid(True, alpha=0.3)
plt.show()

## 4. Multi-channel Processing

Process stereo signals with different content in each channel.

In [ ]:
# Create different signals for left and right channels
left = wd.generate_sin(freqs=[500], duration=1.0, sampling_rate=48000) * 0.05
right = wd.generate_sin(freqs=[2000], duration=1.0, sampling_rate=48000) * 0.1

# Combine into stereo (concatenate along channel axis)
import dask.array as da
stereo_data = da.vstack([left.data[0], right.data[0]])
stereo = wd.ChannelFrame(data=stereo_data, sampling_rate=48000)

# Calculate loudness
loudness = stereo.loudness_zwtv()

print(f"Left channel (500 Hz, lower amp): {loudness[0].mean():.2f} sones")
print(f"Right channel (2000 Hz, higher amp): {loudness[1].mean():.2f} sones")

In [ ]:
# Plot both channels
loudness.plot(overlay=True, title="Stereo Loudness Comparison")
plt.legend(["Left (500 Hz)", "Right (2000 Hz)"])
plt.show()

## 5. Comparison with MoSQITo Direct Call

Verify that wandas results match MoSQITo's direct calculation.

In [ ]:
from mosqito.sq_metrics.loudness.loudness_zwtv import loudness_zwtv as mosqito_loudness

# Create signal
signal = wd.generate_sin(freqs=[1000], duration=1.0, sampling_rate=48000)
signal = signal * 0.063

# Calculate using wandas
loudness_wandas = signal.loudness_zwtv()

# Calculate using MoSQITo directly
data = signal.data[0].compute()
N_mosqito, N_spec, bark_axis, time_axis = mosqito_loudness(
    data, signal.sampling_rate, field_type="free"
)

# Compare
loudness_wandas_data = loudness_wandas.data[0].compute()
difference = np.abs(loudness_wandas_data - N_mosqito)

print(f"Wandas result shape: {loudness_wandas_data.shape}")
print(f"MoSQITo result shape: {N_mosqito.shape}")
print(f"Max difference: {np.max(difference):.10f}")
print(f"Mean difference: {np.mean(difference):.10f}")
print("✓ Results match!" if np.max(difference) < 1e-10 else "✗ Results differ")

## Summary

This notebook demonstrated:

1. **Basic usage**: Calculate loudness for simple signals
2. **Field type comparison**: Compare free field vs diffuse field
3. **Amplitude dependency**: How loudness scales with amplitude
4. **Multi-channel processing**: Process stereo signals independently
5. **Verification**: Confirm results match MoSQITo directly

### Key Points

- Loudness is measured in **sones** (perceptual unit)
- Time resolution is approximately **2ms**
- Each channel is processed **independently**
- Results match **MoSQITo** exactly
- Supports both **free** and **diffuse** field types

### References

- ISO 532-1:2017: "Acoustics — Methods for calculating loudness — Part 1: Zwicker method"
- MoSQITo documentation: https://mosqito.readthedocs.io/en/latest/